In [3]:
import torch
print(torch.__version__)

2.5.1


In [4]:
# Is MPS even available? macOS 12.3+
print(torch.backends.mps.is_available())

# Was the current version of PyTorch built with MPS activated?
print(torch.backends.mps.is_built())

True
True


In [5]:
import tokenizers
print(tokenizers.__version__)

0.19.1


In [2]:
# Import necessary libraries
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from evaluate import load
import torch
import time

In [ ]:
# Import necessary libraries
import torch
import os
import time

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from evaluate import load


load_dotenv()

huggin_face_token = os.getenv("HUGGING_FACE_TOKEN")
# Step 1: Authenticate (if required)

login(huggin_face_token)

# Step 2: Load the Llama 3.2 Model and Tokenizer
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Step 3: Load the Evaluation Dataset
# Here, we use a sample dataset, but you can replace it with your custom dataset as needed
dataset = load_dataset("squad_v2", split="validation")  # Example dataset for QA

# Step 4: Load Metrics
bleu = load("bleu")
rouge = load("rouge")
bertscore = load("bertscore")

# Step 5: Define the Evaluation Function
def evaluate_model(model, tokenizer, dataset):
    references, predictions = [], []
    latencies = []  # To store latency times for each response
    for item in dataset:
        # Start the latency timer
        start_time = time.time()

        # Tokenize the input
        inputs = tokenizer(item["question"], return_tensors="pt").input_ids
        
        # Generate a response from the model
        with torch.no_grad():
            outputs = model.generate(inputs, max_length=50)

        # End the latency timer and calculate elapsed time  
        latency = time.time() - start_time
        latencies.append(latency)

        # Decode the model's output
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Store predictions and references for metric calculation
        predictions.append(prediction)
        references.append(item["answers"]["text"][0] if item["answers"]["text"] else "")

        
    # Calculate average latency
    avg_latency = sum(latencies) / len(latencies)

    # Calculate metrics
    bleu_score = bleu.compute(predictions=predictions, references=references)
    rouge_score = rouge.compute(predictions=predictions, references=references)
    bertscore_score = bertscore.compute(predictions=predictions, references=references, lang="en")

    return {"BLEU": bleu_score, "ROUGE": rouge_score, "BERTScore": bertscore_score, "Average Latency (seconds)": avg_latency}

# Step 6: Run the Evaluation
results = evaluate_model(model, tokenizer, dataset)
print(results)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 